In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import requests
import sys

import tensorflow as tf

tf.random.set_seed(1234)
AUTO = tf.data.experimental.AUTOTUNE

#import tensorflow_datasets as tfds

import os
import re
import numpy as np
from time import time
import matplotlib.pyplot as plt

print("Tensorflow version {}".format(tf.__version__))

In [ ]:


# Maximum number of samples to preprocess
MAX_SAMPLES = 50000

# For tf.data.Dataset
BATCH_SIZE = 64 
BUFFER_SIZE = 20000

# For Transformer
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1

EPOCHS = 10

In [ ]:
import pandas as pd

df_train, df_test = pd.read_csv('atis_intents_train.csv'), pd.read_csv('atis_intents_test.csv')
df_train.columns, df_test.columns = ['intent', 'snippet'], ['intent', 'snippet']

df_train.head()

In [ ]:
df_train.intent.value_counts(), df_train.intent.value_counts(normalize=True)

In [ ]:
train_data = df_train.snippet.values
train_labels = df_train.intent.values
test_data = df_test.snippet.values
test_labels = df_test.intent.values          

print('Training Dataset Size: {}'.format(len(train_data)))
#print('Train Targets: {}'.format(len(train_labels)))
print('Testing Dataset Size: {}'.format(len(test_data)))
#print('Test Targets: {}'.format(len(test_labels)))

In [ ]:
# Maximum sentence length
MAX_LENGTH = 10

tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(list(df_train.snippet) + list(df_test.snippet))
VOCAB_SIZE = tokenizer.document_count
tokenized_requests_train = tokenizer.texts_to_sequences(df_train.snippet)
tokenized_requests_train = tf.keras.preprocessing.sequence.pad_sequences(tokenized_requests_train, 
                                                                    maxlen=MAX_LENGTH,padding='post')
tokenized_requests_test = tokenizer.texts_to_sequences(df_test.snippet)
tokenized_requests_test = tf.keras.preprocessing.sequence.pad_sequences(tokenized_requests_test, 
                                                                    maxlen=MAX_LENGTH,padding='post')

In [ ]:
print('Sample Request: {}'.format(train_data[2]))
print('Sample Token: {}'.format(tokenized_requests_train[2]))
print('Sample Target: {}'.format(train_labels[2]))

In [ ]:
target_train_df = pd.get_dummies(train_labels)
target_test_df = pd.get_dummies(test_labels)
target_train_df.head()

In [ ]:
tokenized_targets_train = pd.get_dummies(train_labels)
tokenized_targets_test = pd.get_dummies(test_labels)
tokenized_targets_train.head()

In [ ]:
tokenized_targets_train = target_train_df.values
tokenized_targets_test = target_test_df.values
print('Dataset Type: {}'.format(type(tokenized_targets_train)))

In [ ]:
# Train/Val Split
prop = 0.8
N = len(tokenized_requests_train)
x_train = tokenized_requests_train[:int(N*prop)]
y_train = tokenized_targets_train[:int(N*prop)]

x_val = tokenized_requests_train[int(N*prop):]
y_val = tokenized_targets_train[int(N*prop):]

x_test = tokenized_requests_test
y_test = tokenized_targets_test

In [ ]:
type(y_test)

In [ ]:
##Implement a Transformer block as a layer
from tensorflow import keras
from tensorflow.keras import layers
#import tensorflow_addons as tfa

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
## Implement embedding layer

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, MAX_LENGTH, VOCAB_SIZE, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=VOCAB_SIZE, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=MAX_LENGTH, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=MAX_LENGTH, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
head_size = 10
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

In [ ]:
inputs = layers.Input(shape=(MAX_LENGTH,))
embedding_layer = TokenAndPositionEmbedding(MAX_LENGTH, VOCAB_SIZE, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(8, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])


In [ ]:
print(x_val.shape)
print(y_train.shape)

In [ ]:
history = model.fit(
    x_train, y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val)
)

In [ ]:
# VALIDATION LOSS curves

plt.clf()
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, (len(history_dict['loss']) + 1))
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:

# VALIDATION ACCURACY curves

plt.clf()
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, (len(history_dict['accuracy']) + 1))
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
print('Sample Request: {}'.format(test_data[2]))
print('Sample Request Token: {}'.format(x_test[2]))
print('Sample Target: {}'.format(test_labels[2]))
print('Sample Target Token: {}'.format(y_test[2]))

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred.shape

In [ ]:
x_test.shape

In [ ]:
y_pred[2,:]